# Reading & visualizaing raw data, cropping, filtering, and saving

## Import the few packages we'll need

In [ ]:
import matplotlib
import pathlib
import mne
import os

Ensure Matplotlib uses the `Qt5Agg` backend, which is the best choice for MNE-Python's interactive plotting functions.

In [ ]:
matplotlib.use('Qt5Agg')

## Load some raw data!

In [ ]:
sample_data_folder = '/Users/christinadelta/datasets/eeg_testing_data'
sample_data_raw_file = os.path.join(sample_data_folder, 'MEG', 'data',
                                    'sample_audvis_raw.fif')

raw = mne.io.read_raw_fif(sample_data_raw_file, preload=True)
# raw = mne.io.read_raw_fif(sample_data_raw_file)

'''
raw.crop(0, 60).load_data() # we'll use the 60 sec of the data for now and load to memory
# load events
events_file = os.path.join(sample_data_folder, 'MEG', 'sample',
                                       'sample_audvis_raw-eve.fif')
events = mne.read_events(events_file)'''

## Let's visualize the raw data!

Spend some time playing around with the interface. Use the help button 

In [ ]:
raw.plot()

## Extract events from the `STIM` channels

to find events in the trigger channels we use the method ```mne.find_events()```. This will give us the total number of events and the event IDs

In [ ]:
events = mne.find_events(raw)
#events 

In [ ]:
event_id = {
    'Auditory/Left': 1,
    'Auditory/Right': 2,
    'Visual/Left': 3,
    'Visual/Right': 4,
    'Smiley': 5,
    'Button': 32
}
event_id

In [ ]:
len(events[events[:, 2] == 32]) # button events 

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
         <li>How many <strong>visual</strong> events are in the data?</li>
    </ul>
</div>

In [ ]:
# To specifically extract the number of visual events only we need to specify that we 
# want the length of events coded as 3 and 4:
print(len(events[events[:, 2]== 3])) # visual left
print(len(events[events[:, 2]== 4])) # visual right

# sum them:
len(events[events[:, 2]== 3]) + len(events[events[:, 2]== 4])

# or..

len(events[events[:, 2]== event_id['Visual/Left']]) + len(events[events[:, 2]== event_id['Visual/Right']])

## Plot the raw data again, but add event markers

In [ ]:
raw.plot(events=events, event_id=event_id)

## Gather some info about the data

Start with the INFO structure

In [ ]:
raw.info

In [ ]:
raw.info['meas_date']

In [ ]:
raw.info['sfreq']

In [ ]:
raw.info['bads']

In [ ]:
raw.ch_names[:10]

In [ ]:
raw.info['chs'][0]

## Visualize the sensor locations

In [ ]:
raw.plot_sensors(ch_type='eeg')

In [ ]:
raw.plot_sensors(kind='3d', ch_type='eeg')

## Mark channels as bad

Mark an additional EEG channel as bad and view the topoplot.

In [ ]:
raw.info['bads']

In [ ]:
raw.info['bads'] += ['EEG 051', 'EEG 050']
raw.plot_sensors(ch_type='eeg')

## Select only a subset of the channels

In [ ]:
raw_eeg = raw.copy().pick_types(meg=False, eeg=True, eog=True, exclude=[])
len(raw_eeg.ch_names)

In [ ]:
raw_eeg.info

In [ ]:
raw_eeg.plot() # plot the eeg data

In [ ]:
raw_eeg.plot(events=events, event_id=event_id) # plot the eef with the events

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
         <li>Select only MEG channels ("meg")</li>
         <li>Select only magnetometer channels ("mag")</li>
    </ul>
</div>

In [ ]:
# add the meg data in a new variable 
raw_meg = raw.copy().pick_types(meg=True, eeg=False, eog=False, exclude=[])
len(raw_meg.ch_names)

raw_meg.plot()

To choose only magnetometer channels we can use the ```.pick_types()``` function again, however, instead of passing a boolean (True, False) to the ```meg``` argument, we pass a string (e.g. ```meg=grad``` to pick only gradiometer channels or ```meg='mag'``` to pick only magnetometer channels) 

In [ ]:
# add the meg-magnetometer data in a new variable 
raw_mag = raw.copy().pick_types(meg='mag', eeg=False, eog=False, exclude=[])
len(raw_mag.ch_names)

raw_mag.plot()

## Crop and filter the data

In [ ]:
raw_eeg_cropped = raw_eeg.copy().crop(tmax=100).load_data()
raw_eeg_cropped.times[-1] # take a look at the latest time-point of the cropped data

If we want to search the documentation with the parameters/arguments that each function accepts, move the cursor in the parenthesis after the function name and press **shift** and **tab** together

In [ ]:
raw_eeg_cropped_filtered = raw_eeg_cropped.filter(l_freq=0.1, h_freq=40)

In [ ]:
raw_eeg_cropped.plot(events=events, event_id=event_id, title='before_filtering')
raw_eeg_cropped_filtered.plot(events=events, event_id=event_id, title='filtered')

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(2)

raw_eeg_cropped.plot_psd(ax=ax[0], show=False)
raw_eeg_cropped_filtered.plot_psd(ax=ax[1], show=False)

ax[0].set_title('PSD before filtering')
ax[1].set_title('PSD after filtering')
ax[1].set_xlabel('Frequency (Hz)')
fig.set_tight_layout(True)
plt.show()

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
         <li>Filter the raw data with a 1 Hz high-pass and a 30 Hz low-pass filter and plot the PSD.</li>
    </ul>
</div>

In [ ]:
raw_eeg_cropped_filtered2 = raw_eeg_cropped.filter(l_freq=1, h_freq=30)

In [ ]:
# plot two PSDs
fig, ax = plt.subplots(2)
raw_eeg_cropped_filtered.plot_psd(ax=ax[0], show=False)
raw_eeg_cropped_filtered2.plot_psd(ax=ax[1], show=False)

ax[0].set_title('PSD first filtering')
ax[1].set_title('PSD second filtering')
ax[1].set_xlabel('Frequency (Hz)')
fig.set_tight_layout(True)
plt.show()

## Save the data

In [ ]:
raw_eeg_cropped_filtered.save(pathlib.Path('out_data') / 'eeg_cropped_filt_raw.fif', 
                              overwrite=True)